In [1]:
import numpy as np
import pandas as pd
from functions import createDataMain,createTestDataMain,pd2np,pd2np_test
import cPickle

In [2]:
batch_size = 100
discreteList = ['dayOfWeek','payDay','month','earthquake',\
                'type','locale','locale_name','transferred','onpromotion']


In [3]:
with open(r"dateVar.pickle", "rb") as input_file:
    dateVar_list = cPickle.load(input_file)

Save item lookup tables

In [5]:
types = {'id': 'int32',
     'item_nbr': 'int32',
     'store_nbr': 'int8',
     'unit_sales': 'float32',
     'onpromotion': bool}
train = pd.read_csv('train.csv',usecols=['date','item_nbr','store_nbr','unit_sales','onpromotion'],\
                    parse_dates=['date'],dtype=types, infer_datetime_format=True)
train = train.fillna(2,axis=1)
train.onpromotion = train.onpromotion.astype(np.int8)
train.loc[train.unit_sales<0,'unit_sales'] = .0 # clip negative sales to zero
item_uniq = pd.unique(train.item_nbr)
item_dict = {item:i+1 for i,item in enumerate(item_uniq)}

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
with open(r"item_dict_final.pickle", "wb") as output_file:
    cPickle.dump(item_dict, output_file)

In [7]:
train = train[train.date < '2017-07-31']
item_uniq = pd.unique(train.item_nbr)
item_dict = {item:i+1 for i,item in enumerate(item_uniq)}

In [8]:
with open(r"item_dict_train.pickle", "wb") as output_file:
    cPickle.dump(item_dict, output_file)

*** Train and Validation datasets ***

In [4]:
train,val = createDataMain(True)

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [5]:
y_np, weight_np,Con_np,Dis_np,X_np,Count_np = \
            pd2np(val,batch_size,val.countDays.max(),dateVar_list,'date',discreteList)

In [6]:
prefix = 'val'

In [7]:
np.savetxt(prefix+'_Y',y_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Weight',weight_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Count',Count_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")  

In [8]:
y_np, weight_np,Con_np,Dis_np,X_np,Count_np = \
            pd2np(train,batch_size,train.countDays.max(),dateVar_list,'date',discreteList)

In [9]:
sorts = np.argsort(Count_np)
y_np, weight_np,Con_np,Dis_np,X_np,Count_np = y_np[sorts], weight_np[sorts],\
                                            Con_np[sorts],Dis_np[sorts],X_np[sorts],Count_np[sorts]

In [10]:
prefix = 'train'

In [11]:
np.savetxt(prefix+'_Y',y_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Weight',weight_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Count',Count_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")  

y_np = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

*** Test datasets ***

In [5]:
test_SI,test_newItem = createTestDataMain(True)

In [26]:
y_np, Con_np,Dis_np,X_np = \
pd2np_test(test_SI,batch_size,16,dateVar_list,'date',discreteList)

In [27]:
prefix = 'test_SI'

In [28]:
np.savetxt(prefix+'_Y',y_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")  

In [29]:
y_np, Con_np,Dis_np,X_np = \
pd2np_test(test_newItem,batch_size,16,dateVar_list,'date',discreteList)

In [30]:
prefix = 'test_newItem'

In [31]:
np.savetxt(prefix+'_Y',y_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")  

In [16]:
# train[index] gives the corresponding historical Xs for test dataset
index = test_SI.merge(train.reset_index()[['index','store_nbr','item_nbr']],\
                     'left',['store_nbr','item_nbr'])['index'].values.astype(np.int32)

In [23]:
np.savetxt('Index',index,fmt="%d",delimiter=",") 